## Smoothing

Smoothing can help to discover trends that otherwise might be hard to see in raw data. 

In [1]:
%useLatestDescriptors
%use lets-plot
%use ggdsl(0.1.4-dev-23)

In [2]:
LetsPlot.getInfo()  // This prevents Krangl from loading an obsolete version of Lets-Plot classes.

Lets-Plot Kotlin API v.4.0.0. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.2.4.0.

In [5]:
%use dataframe

In [6]:
var mpg_df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/data/mpg.csv")
mpg_df.head()


DataFrame: rowsCount = 5, columnsCount = 12

### The default smoothing method is `'linear model'` (or `'lm'`)

In [7]:
// val dat = (mpg_df.names.map { Pair(it, mpg_df.get(it).values())}).toMap()
val dat = mpg_df.toMap()

In [8]:
val mpg_plot = letsPlot(dat) {x="displ"; y="hwy"} 
mpg_plot + geomPoint() + geomSmooth()

In [11]:
mpg_df.create { plot {
    points { 
        x(displ)
        y(hwy)
    }
    smooth(displ, hwy) {}
} }

### `LOESS` model does seem to better fit MPG data than linear model.

In [9]:
mpg_plot + geomPoint() + statSmooth(method="loess", size=1.0)

In [13]:
mpg_df.create { plot {
    points { 
        x(displ)
        y(hwy)
    }
    smooth(displ, hwy, method = SmoothMethod.Loess()) {
        lineWidth(1.0)
    }
} }

### Applying smoothing to groups

Let's map the vehicle `drivetrain type` (variable 'drv') to the color of points.

This makes it easy to see that points with the same type of the drivetrain are forming some kind of groups or clusters. 

In [14]:
mpg_plot + geomPoint {color="drv"} +
           statSmooth(method="loess", size=1.0) {color="drv"}

In [15]:
mpg_df.create { plot {
    points { 
        x(displ)
        y(hwy)
        color(drv)
    }
    smooth(displ, hwy, method = SmoothMethod.Loess()) {
        lineWidth(1.0)
        lineColor(drv)
    }
} }

### Apply linear model with 2nd degree polynomial.

As `LOESS` prediction looks a bit weird let's try 2nd degree polinomial regression.

In [16]:
mpg_plot + geomPoint {color="drv"} +
           statSmooth(method="lm", deg=2, size=1.0) {color="drv"}

In [17]:
mpg_df.create { plot {
    points { 
        x(displ)
        y(hwy)
        color(drv)
    }
    smooth(displ, hwy, method = SmoothMethod.Linear(degree = 2)) {
        lineWidth(1.0)
        lineColor(drv)
    }
} }

## Using `asDiscrete()` function with numeric data series

In the previous examples we were using a discrete (or categorical) variable `'drv'` to split the data into a groups.

Now let's try to use a numeric variable `'cyl'` for the same purpose.

In [18]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0) {color="cyl"}

In [19]:
mpg_df.create { plot {
    points { 
        x(displ)
        y(hwy)
        color(cyl)
    }
    smooth(displ, hwy, method = SmoothMethod.Linear(degree = 2)) {
        lineWidth(1.0)
        lineColor(cyl)
    }
} }

Easy to see that the data wasn't split into groups. 
`Lets-Plot` offers two solutions in this situation:

 * Use the `group` aesthetic
 * Use the `asDiscrete()` function

The **group** aesthetic helps to create a groups.

In [20]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0) {color="cyl"; group="cyl"}

In [21]:
mpg_df.create { plot {
    points { 
        x(displ)
        y(hwy)
        color(cyl)
    }
    smooth(displ, hwy, method = SmoothMethod.Linear(degree = 2)) {
        lineWidth(1.0)
        lineColor(cyl)
        splitBy(cyl)
    }
} }

The **asDiscrete('cyl')** function will "annotate" the `'cyl'` variable as `discrete`.

This leads to creation of the groups and to assigning of a `discrete` color scale instead of a `continuous`.

In [24]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0) {color=asDiscrete("cyl")}

In [23]:
mpg_df.create { plot {
    points { 
        x(displ)
        y(hwy)
        color(cyl)
    }
    smooth(displ, hwy, method = SmoothMethod.Linear(degree = 2)) {
        lineWidth(1.0)
        lineColor(cyl.scaled(categorical()))
    }
} }

## Effect of `span` parameter on the "wiggliness" the LOESS smoother.

The span is the fraction of points used to fit each local regression.
Small numbers make a wigglier curve, larger numbers make a smoother curve.

In [25]:
import kotlin.math.PI
import kotlin.random.Random


In [29]:
val n = 150
val x_range = generateSequence( -2 * PI ) { it + 4 * PI / n }.takeWhile { it <= 2 * PI }
val y_range = x_range.map{ sin( it ) + Random.nextDouble(-0.5, 0.5) }
val df = mapOf<String, List<Any>>(
    "x" to x_range.toList(),
    "y" to y_range.toList()
)

In [27]:
val p = ggplot(df) {x="x"; y="y"} + geomPoint(shape=21, fill="yellow", color="#8c564b")
val p1 = p + geomSmooth(method="loess", size=1.5, color="#d62728") + ggtitle("default (span = 0.5)")
val p2 = p + geomSmooth(method="loess", span=.2, size=1.5, color="#9467bd") + ggtitle("span = 0.2")
val p3 = p + geomSmooth(method="loess", span=.7, size=1.5, color="#1f77b4") + ggtitle("span = 0.7")
val p4 = p + geomSmooth(method="loess", span=1, size=1.5, color="#2ca02c") + ggtitle("span = 1")

GGBunch()
.addPlot(p1, 0, 0, 400, 300)
.addPlot(p2, 400, 0, 400, 300)
.addPlot(p3, 0, 300, 400, 300)
.addPlot(p4, 400, 300, 400, 300)

In [28]:
val xSrc = "x"<Double>()
val ySrc = "y"<Double>()

In [30]:
val pA = plot(df) {
    points { 
        x(xSrc)
        y(ySrc)
        symbol(Symbol.CIRCLE_FILLED)
        fillColor(Color.YELLOW)
        color(Color.fromHex("#8c564b"))
    }
    
    smooth(xSrc, ySrc, method = SmoothMethod.Loess()) {
        lineColor(Color.fromHex("#d62728"))
        lineWidth(1.5)
    }
    layout.size = "default (span = 0.5)"
}
plotBunch { 
    add(pA, 0, 0, 400, 300)
}